In [2]:
# алгоритм из статьи П.B.Шпилева 
# Алгоритм 4.1. ПостРОЕНИЕ D-ОПТИМАЛЬНОГО ПЛАНА
# Шаг 1: В качестве начальной точки выбирается план из Теоремы 4.1 ,являющийся D-оптимальным на гиперпараллелепипеде X’( его границы а_j выбираются таким образом, чтобы минимизировать количество требуемых преобразований для перехода от X’ к X). Устанавливаются значения а_j_1, .. а_j_k (скоростей адаптации).

# Шаг 2: Границы области X’ сдвигаются на аа_j_1, .. а_j_k в направлении соответствующих границ X (для каждого отрезка, [а_i_1 , а_i_2|). Пересчитываются точки и веса плана
# (для численного нахождения оптимального плана применяется какой-либо оптимизационный алгоритм (алгоритм CMA ES). 

# Шаг 3: Найденный план проверяется на оптимальность с помощью теоремы (2.1). В случае положительного результата проверки, множество точек носителя найденного оптимального плана сравнивается с множеством корней уравнения
# экстремальный полином плана - p = 0) и, если обнаруживается, что эти множества не совпадают, то в качестве начального приближения при следующей итерации выбирается 3 плана:текущий план;  план, точки носителя которого дополнены точками из множества B с нулевыми весами и план, в носителе которого точки с минимальными весами заменены на точки из множества B, где B - множество максимумов экстремального полинома, которые не являются точками носителя плана. В случае, если B является пустым множеством, все эти три плана совпадают. Для этих планов выполняются шаги 2 и 3 до тех пор, пока Х не равно X’,
# после чего выполняется шаг 5. В противном случае — шаг 4.

# Шаг 4: Устанавливается пороговый уровень е > 0. Выбирается последний найденный оптимальный план, если все его веса, больше е, выбираются новые скорости адаптации а_j+1 < а_j; и b_j+1 < b_j, выполняются шаги 2 и 3. В противном случае, из плана
# исключаются точки с весом меньше e, устанавливаются новые скорости адаптации (с учетом оставшихся расстояний между соответствующими границами областей Х
# и X’) и выполняются шаги 2 и 3.

# Шаг 5: Проверяем, что экстремальный полином в точках оптимального плана имеет достигает своего глобального максимума, и при необходимости исключаем из плана,
# избыточные точки. Завершаем работу алгоритма.

# Данный алгоритм позволяет гарантировать, что численное решение целевой оптимизационной задачи (3) сойдется к глобальному максимуму. Скорости адаптации и пороговый уровень являются настраиваемыми параметрами алгоритма.

In [20]:
import numpy as np
from scipy.optimize import root_scalar, minimize
from cma import CMAEvolutionStrategy  # библиотека для CMA-ES

In [6]:
def generate_plan(a, k):
    """Генерация точек плана."""
    vertices = []
    edges = []
    for combo in np.array(np.meshgrid(*[[-a_j, a_j] for a_j in a])).T.reshape(-1, k):
        vertices.append(combo)
    for i in range(k):
        for combo in np.array(np.meshgrid(*[[-a_j, a_j] if i != j else [0] for j, a_j in enumerate(a)])).T.reshape(-1, k):
            edges.append(combo)
    return np.array(vertices), np.array(edges)


In [7]:
def optimize_plan(vertices, edges, k, func):
    """Оптимизация CMA-ES."""
    initial_guess = np.random.rand(k)  # начальное приближение
    es = CMAEvolutionStrategy(initial_guess, 0.5)  # стратегия CMA-ES
    result = es.optimize(func)
    return result.result.xbest


In [8]:
def main(k, a):
    w1 = find_w1(k)
    if w1 is None:
        raise ValueError("Failed to find w1.")
    vertices, edges = generate_plan(a, k)
    # Оптимизация и проверка
    optimized_plan = optimize_plan(vertices, edges, k, your_optimization_function)
    return optimized_plan


In [ ]:
import numpy as np
from scipy.optimize import root_scalar, minimize
from cma import CMAEvolutionStrategy  # библиотека для CMA-ES


def find_w1(k):
    coeffs = [2 * k ** 2 + 5 * k - 3, -3 * k ** 2 - 8 * k + 5, k ** 2 + 5 * k - 4, 1 - k]
    root = root_scalar(lambda w: np.polyval(coeffs, w), bracket=(0, 1 / k))
    return root.root if root.converged else None


def phi(x):
    # Вектор базисных функций для квадратичной модели
    k = len(x)
    phi_x = []
    phi_x.extend([xi ** 2 for xi in x])  # квадратичные члены
    for i in range(k):
        for j in range(i + 1, k):
            phi_x.append(x[i] * x[j])  # смешанные члены
    phi_x.extend([xi for xi in x])  # линейные члены
    phi_x.append(1)  # свободный член
    return np.array(phi_x)


# Функция для нормализации весов (приведение к сумме 1)
def normalize_weights(weights):
    return weights / np.sum(weights)


def fisher_matrix(points, weights):
    """
    Вычисляет матрицу Фишера на основе точек и весов.
    Args:
        points (list of np.ndarray): Точки плана, например, [np.array([x1, y1]), ...].
        weights (list of float): Веса, например, [w1, w2, ...], сумма весов = 1.
        model_gradient (callable): Функция, вычисляющая градиент модели по параметрам.
                                   model_gradient(point) -> np.ndarray

    Returns:
        np.ndarray: Матрица Фишера.
    """
    weights = normalize_weights(weights)  # Нормализуем веса
    n_features = len(phi(points[0]))
    M = np.zeros((n_features, n_features))
    for point, weight in zip(points, weights):
        phi_x = phi(point)
        M += weight * np.outer(phi_x, phi_x)
    return M


def log_determinant(variables, num_points, k):
    """
    Вычисляет лог-определитель матрицы Фишера.
    variables: массив всех неизвестных (координаты точек и веса).
    num_points: количество точек плана.
    k: размерность области поиска
    """
    points = [variables[i:i + k] for i in range(0, num_points * k, k)]
    weights = variables[num_points * k:]

    fisher_matrix = fisher_matrix(points, weights)
    det = np.linalg.det(fisher_matrix)
    if det <= 0:
        return -np.inf  # Избегаем некорректного логарифма
    return np.log(det)


# Инвертируем функцию для минимизации (CMA-ES минимизирует по умолчанию)
def negative_log_determinant(variables, num_points, k):
    return -log_determinant(variables, num_points, k)


def extreme_function(x, M_inv):
    """Экстремальная функция p(x) = m - φ(x)^T M^-1 φ(x)."""
    m = len(phi(x))
    phi_x = phi(x)
    return m - phi_x.T @ M_inv @ phi_x


def check_extreme_function(points, weigths, e=1e-6):
    inverse_fisher = fisher_matrix_inverse(points, weights)
    for point in points:
        if abs(extreme_function(point, inverse_fisher)) > e:
            return false
    return true


def fisher_matrix_inverse(points, weights, e=1e12):
    """
    Вычисляет обратную матрицу Фишера на основе точек и весов.
    Args:
        points (list of np.ndarray): Точки плана.
        weights (list of float): Веса (сумма весов должна равняться 1).
        e(float): Порог обуcловленности.
    Returns:
        np.ndarray: Обратная матрица Фишера.
    """
    M = fisher_matrix(points, weights)

    if np.linalg.cond(M) > e:
        raise ValueError("Матрица Фишера плохо обусловлена или вырожденная.")

    return np.linalg.inv(M)


def get_null_weights(weights, e=1e-3):
    nums = []
    for i, weight in enumerate(weights):
        if abs(weight) < e:
            nums.append(i)
    return nums


def check_local_maxima(points, M_inv, m, neighborhood_size=0.1, num_samples=10):
    """
    Проверяет, являются ли точки плана локальными максимумами экстремальной функции.
    """
    for point in points:
        neighborhood = np.random.uniform(
            low=point - neighborhood_size,
            high=point + neighborhood_size,
            size=(num_samples, len(point))
        )

        p_values = [extreme_function(x, M_inv) for x in neighborhood]
        if any(p > m for p in p_values):
            print(f"Point {point} is not a local maximum.")
            return False
    print("All points are local maxima.")
    return True


def global_max_check(M_inv, bounds, m):
    """
    Проверяет, что глобальный максимум экстремальной функции не превышает m.
    """
    dim = len(bounds)

    # Задаем целевую функцию для глобальной оптимизации
    def objective(x):
        return -extreme_function(x, M_inv)

    # Используем библиотеку PyGMO для глобальной оптимизации
    prob = pg.problem(pg.continuous(objective, dim, bounds))
    algo = pg.algorithm(pg.de(gen=100))  # Дифференциальная эволюция
    pop = pg.population(prob, 50)  # Начальная популяция
    pop = algo.evolve(pop)
    best_f = -pop.champion_f  # Находим максимум (с учетом инверсии)

    print(f"Global maximum of p(x) = {best_f}")
    return best_f <= m

# Пример использования
k = 2  # размерность задачи
# Начальная несимметричная область b[0] x b[1]
b = [(-1, 1),(1.5, 8)]

alpha = []
new_b = []
for pair in b:
    if abs(pair[0]) != abs(pair[1]):
        if abs(pair[0]) > abs(pair[1]):
            new_b.append((pair[0], abs(pair[0])))
            alpha.append((abs(pair[0]) - pair[1]) / 2)
        else:
            new_b.append((-pair[1], pair[1]))
            alpha.append((pair[0] + pair[1]) / 2)
    else:
        new_b.append(pair)
        alpha.append(0)
print("new_b =", new_b)
print("alpha =", alpha)

w1 = find_w1(k)
w2 = 1/2**k - k/2*w1
print(f"w1 = {w1}, w2 = {w2}")
# Ближайшая симметричная область, включающая a2 - [-8, 8], у нее такие точки плана и веса, согласно Теореме 4.1
num_points = k*2**(k-1)+2**k  # Количество точек плана
initial_points = np.array([
    [0, -8], [-8, 0], [8, 0], [0, 8], [8, -8], [-8, 8], [-8, -8], [8, 8]
])
initial_weights = [w1, w1, w1, w1, w2, w2, w2, w2]


#максимизируем лог определителя матрицы Фишера
sigma = 0.5  # Начальное стандартное отклонение
# Преобразуем начальные точки и веса в плоский массив для оптимизации
initial_variables = np.hstack([initial_points.flatten(), initial_weights])
# формируем новую область поиска
new_field = []
for i, a in enumerate(alpha):
    if a == 0:
        new_field.append(b[i])
    else:
        f = []
        for j in range(k):
            if new_b[i][j] != b[i][j]:
                f.append(new_b[i][j] + a)
            else:
                f.append(b[i][j])
        new_field.append(tuple(f))
print("new_field =", new_field)

# Запуск CMA-ES
bounds = [[new_field[0][0], new_field[1][0]] * num_points, [new_field[0][1], new_field[1][1]] * num_points, [0, 1] * num_points]  # Минимальные и максимальные границы для каждой переменной
es = CMAEvolutionStrategy(initial_variables, sigma, {'bounds': bounds})
result = es.optimize(lambda vars: negative_log_determinant(vars, num_points, k))

# Получение результатов
optimal_variables = result.result.xbest
optimal_points = [optimal_variables[i:i+k] for i in range(0, num_points * k, k)]
optimal_weights = normalize_weights(optimal_variables[num_points * k:])
optimal_value = -result.result.fbest

if not check_extreme_function(optimal_points, optimal_weights):
    nums = get_null_weights(optimal_weights)
